In [1]:
pip install pandas numpy matplotlib seaborn jupyterlab sqlalchemy psycopg2-binary

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ------------------- -------------------- 1.0/2.1 MB 7.0 MB/s eta 0:00:01
   ----------------------------- ---------- 1.6/2.1 MB 4.4 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 4.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ------------ --------------------------- 0.8/2.6 MB 4.8 MB/s eta 0:00:01
   ---------------- ----------------------- 1.0/2.6 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 4.0 MB/s eta 0:00:00

   ---------------------------------------- 0/3 [psycopg2-binary]
   ---------------------------------------- 0/3 [psycopg2-binary]
   ---------------------------------------- 0/3 [psycopg2-binary]
   ------------- -------------------------- 1/3 [greenlet]
   ------------- -------------------------- 1/3 [greenlet]
   ------------- -------------------------- 1/3 [greenlet]
   ------------- -------------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import timedelta
from urllib.parse import quote_plus

In [ ]:
print("Connecting to database to fetch data for analysis...")

# --- 1. CONNECT AND FETCH DATA ---
db_user = "postgres"
db_password_raw = "Abhin@vFaldu9"
db_host = "localhost"
db_port = "5432"
db_name = "ecommerce_db"
table_name = "superstore"
db_password_encoded = quote_plus(db_password_raw)

engine = create_engine(
    f"postgresql://{db_user}:{db_password_encoded}@{db_host}:{db_port}/{db_name}"
)

try:
    df = pd.read_sql_table(table_name, engine)
    print("Successfully fetched data.")
except Exception as e:
    print(f"An error occurred: {e}")
    exit()

Connecting to database to fetch data for analysis...
Successfully fetched data.


In [ ]:
print("\nPerforming RFM customer segmentation...")

current_date = df["order_date"].max() + timedelta(days=1)

rfm = (
    df.groupby("customer_id")
    .agg(
        {
            "order_date": lambda x: (current_date - x.max()).days,  
            "order_id": "nunique",  
            "sales": "sum",  
        }
    )
    .round(2)
)

rfm.columns = ["recency", "frequency", "monetary"]


rfm["r_score"] = pd.qcut(rfm["recency"].rank(method="first"), 5, labels=[5, 4, 3, 2, 1])
rfm["f_score"] = pd.qcut(
    rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5]
)
rfm["m_score"] = pd.qcut(
    rfm["monetary"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5]
)


rfm["rfm_score"] = (
    rfm["r_score"].astype(str) + rfm["f_score"].astype(str) + rfm["m_score"].astype(str)
)



def segment_customers(row):
    if row["r_score"] >= 4 and row["f_score"] >= 4:
        return "Champions"
    elif row["r_score"] >= 3 and row["f_score"] >= 3:
        return "Loyal Customers"
    elif row["r_score"] >= 3 and row["f_score"] < 3:
        return "Potential Loyalists"
    elif row["r_score"] < 3 and row["f_score"] >= 3:
        return "At Risk"
    elif row["r_score"] < 2:
        return "Hibernating"
    else:
        return "New Customers"


rfm["customer_segment"] = rfm.apply(segment_customers, axis=1)


segment_summary = (
    rfm.groupby("customer_segment")
    .agg({"recency": "mean", "frequency": "mean", "monetary": ["mean", "count"]})
    .round(2)
)

print("\n📊 Customer Segment Performance:")
print(segment_summary)



Performing RFM customer segmentation...

📊 Customer Segment Performance:
                    recency frequency monetary      
                       mean      mean     mean count
customer_segment                                    
At Risk              220.23      7.49  3407.51   146
Champions             25.48      8.98  4047.00   167
Hibernating          494.81      3.50  1695.18   113
Loyal Customers       59.12      7.26  3277.73   163
New Customers        149.10      4.12  1662.55    58
Potential Loyalists   45.22      4.10  2065.77   146


In [ ]:
df_with_rfm = pd.merge(df, rfm, on="customer_id", how="left")

output_filename = "superstore_with_rfm.csv"
df_with_rfm.to_csv(output_filename, index=False)

print(
    f"\nAnalysis complete. Full dataset with RFM segments saved to '{output_filename}'"
)


Analysis complete. Full dataset with RFM segments saved to 'superstore_with_rfm.csv'
